<a href="https://colab.research.google.com/github/Harini-que/Care_Buddy/blob/main/facebiomet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- Install dependencies ---
!pip install deepface fer mtcnn opencv-python-headless ipywidgets

import cv2
import numpy as np
import base64
import time
import pickle
import os
from PIL import Image as PILImage
from IPython.display import display, Image as DisplayImage, clear_output
from google.colab.output import eval_js
import ipywidgets as widgets
from deepface import DeepFace
from fer import FER

# ----------------- DATABASE -----------------
DB_PATH = "user_db_multi.pkl"

def load_user_db():
    if os.path.exists(DB_PATH):
        with open(DB_PATH, "rb") as f:
            db = pickle.load(f)
        print("✅ User database loaded:", list(db.keys()))
    else:
        db = {}
        print("ℹ️ No user database found. Starting fresh.")
    return db

user_db = load_user_db()

def register_user(name, image_paths):
    embeddings = []
    for img_path in image_paths:
        try:
            emb = DeepFace.represent(img_path, enforce_detection=True, model_name="Facenet512")[0]["embedding"]
            embeddings.append(emb)
        except:
            print(f"⚠️ Face not detected in {img_path}, skipping...")
    if embeddings:
        user_db[name] = embeddings
        with open(DB_PATH, "wb") as f:
            pickle.dump(user_db, f)
        print(f"✅ User '{name}' registered with {len(embeddings)} valid images!")
    else:
        print("❌ No valid face images provided. Registration failed.")

# ----------------- WEBCAM CAPTURE -----------------
def capture_webcam():
    js_code = """
    async function capture() {
      const video = document.createElement('video');
      video.width = 320; video.height = 240;
      document.body.appendChild(video);
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      video.srcObject = stream;
      await video.play();
      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0, canvas.width, canvas.height);
      const data = canvas.toDataURL('image/jpeg', 0.8);
      stream.getTracks().forEach(track => track.stop());
      video.remove();
      return data;
    }
    capture();
    """
    data = eval_js(js_code)
    encoded_data = data.split(',')[1]
    nparr = np.frombuffer(base64.b64decode(encoded_data), np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (640, 480))
    cv2.imwrite("captured_frame.jpg", img)
    return img

# ----------------- VERIFICATION -----------------
def verify_user(captured_image_path, threshold=0.35):
    img_gray = cv2.imread(captured_image_path, cv2.IMREAD_GRAYSCALE)
    if np.mean(img_gray) < 30:
        print("❌ Image too dark or blank. Capture again.")
        return None
    try:
        cap_emb = DeepFace.represent(captured_image_path, enforce_detection=True, model_name="Facenet512")[0]["embedding"]
    except:
        print("❌ Face not detected properly. Try again.")
        return None

    best_match, best_similarity = None, -1
    for name, embeddings in user_db.items():
        for emb in embeddings:
            cos_sim = np.dot(emb, cap_emb) / (np.linalg.norm(emb) * np.linalg.norm(cap_emb))
            if cos_sim > best_similarity:
                best_similarity = cos_sim
                best_match = name

    if best_similarity >= threshold:
        print(f"✅ Authentication Successful! User: {best_match} (Similarity: {best_similarity:.3f})")
        return best_match
    else:
        print(f"❌ Authentication Failed! Best similarity: {best_similarity:.3f}")
        return None

# ----------------- EMOTION DETECTION -----------------
# ----------------- EMOTION DETECTION -----------------
detector = FER(mtcnn=False)  # ✅ OpenCV detector, robust for Colab
NUM_FRAMES = 7
FRAME_DELAY = 0.2

def detect_emotion_webcam():
    print("\n🎭 Capturing frames for emotion detection...")
    emotions_accum = []

    for i in range(NUM_FRAMES):
        frame = capture_webcam()
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # ✅ Directly pass NumPy array (OpenCV) to FER
        results = detector.detect_emotions(frame_rgb)
        if results:
            emotions_accum.append(results[0]["emotions"])
        else:
            # No face detected, use zeros
            emotions_accum.append({e:0 for e in ["angry","disgust","fear","happy","sad","surprise","neutral"]})

        display(DisplayImage(filename="captured_frame.jpg"))
        time.sleep(FRAME_DELAY)

    avg_emotions = {emo: np.mean([e[emo] for e in emotions_accum]) for emo in emotions_accum[0].keys()}
    dominant_emotion = max(avg_emotions, key=avg_emotions.get)
    confidence = avg_emotions[dominant_emotion]

    print(f"\n✅ Dominant Emotion: {dominant_emotion} (Confidence: {confidence:.2f})")
    return dominant_emotion, confidence


# ----------------- UI MODULE -----------------
reg_output = widgets.Output()
display(reg_output)

upload_btn = widgets.FileUpload(accept='image/*', multiple=True)
with reg_output:
    display(upload_btn)

def on_upload_change(change):
    if len(upload_btn.value) > 0:
        image_paths = []
        for fname, file_info in upload_btn.value.items():
            with open(fname, "wb") as f:
                f.write(file_info['content'])
            image_paths.append(fname)

        name_box = widgets.Text(description="User Name:")
        confirm_btn = widgets.Button(description="Register User")
        display(name_box, confirm_btn)

        def on_confirm(b):
            if name_box.value.strip() == "":
                print("⚠️ Please enter a valid user name!")
                return
            register_user(name_box.value.strip(), image_paths)
            name_box.close()
            confirm_btn.close()
            upload_btn.close()
            print("✅ Registration complete! You can now Capture & Verify.")

        confirm_btn.on_click(on_confirm)

upload_btn.observe(on_upload_change, names='value')

# --- Capture & Verify Button ---
capture_btn = widgets.Button(description="Capture & Verify")
output = widgets.Output()
display(capture_btn, output)

def on_capture_click(b):
    with output:
        clear_output()
        frame = capture_webcam()
        display(DisplayImage(filename="captured_frame.jpg"))
        user = verify_user("captured_frame.jpg")
        if user:
            print("✅ You can now run emotion detection separately.")

capture_btn.on_click(on_capture_click)

# --- Detect Emotion Button ---
emotion_btn = widgets.Button(description="Detect Emotion")
display(emotion_btn)

def on_emotion_click(b):
    with output:
        clear_output()
        detect_emotion_webcam()

emotion_btn.on_click(on_emotion_click)

✅ User database loaded: ['Afraah']


Output()

Button(description='Capture & Verify', style=ButtonStyle())

Output()

Button(description='Detect Emotion', style=ButtonStyle())